In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
from nitools.firstlevel import Dataset
from nistats.hemodynamic_models import glover_hrf

/opt/conda/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


## Define analysis code
Some custom classes.

## Define data+task set
Corresponding BIDS-dir, subject-identifier, etc. etc.

In [5]:
ds = Dataset(bids_dir='/home/lsnoek1/spinoza_data/ni-edu/bids', sub='05', n_jobs=1)
ds.create_taskset(
    task='flocBLOCKED',
    space='fsaverage6',
    tr=0.7,
    acq=None,
    ses=True,
    hemi='R',
    reference_run=-1,
    use_gm_mask=True,
    gm_threshold=0.5
)

Found 2 complete runs for task flocBLOCKED


## Check floc (BLOCKED) localizer

In [ ]:
ds.flocBLOCKED.preprocess(
    df_filter=None,
    regress_confounds=False,
    conf_vars=None,
    smoothing_fwhm=3,
    hp_cutoff=100
)

In [ ]:
ds.flocBLOCKED.glm_detect(hrf_model='glover')
floc_mask = ds.flocBLOCKED.compute_fxe('4*face - object - place - body - character', stat_type='t')
ds.visualize(floc_mask, space='fsaverage6', threshold=4)

In [ ]:
ds.flocBLOCKED.plot_design(exclude_confs=True)

In [ ]:
import numpy as np
from scipy.stats import pearsonr

dm2 = ds.flocBLOCKED.glm['dms'][0]
dmfsl = np.loadtxt('sub-03old/ses-1.feat/design.mat', skiprows=5)

fig, ax = plt.subplots(nrows=12, figsize=(15, 15))
for i, col in enumerate(['face', 'body', 'place', 'character', 'object', 'response',
                         'rot_x', 'rot_y', 'rot_z', 'trans_x', 'trans_y', 'trans_z']):
    print(pearsonr(dm2.loc[:, col].values, dmfsl[:, i]))
    ax[i].plot(dm2.loc[:, col].values)
    ax[i].plot(dmfsl[:, i])

## Check floc (ER) HRF mapper

In [ ]:
ds.create_taskset(
    task='flocER',
    space='fsaverage6',
    tr=0.7,
    acq=None,
    ses=True,
    hemi='R',
    reference_run=-1,
    use_gm_mask=True,
    gm_threshold=0.5
)

In [ ]:
ds.flocER.preprocess(
    df_filter=None,
    regress_confounds=False,
    conf_vars=None,
    smoothing_fwhm=5,
    hp_cutoff=100
)

In [ ]:
ds.flocER.glm_detect(hrf_model='glover')
floc_mask = ds.flocER.compute_fxe('4*face - object - place - body - character', stat_type='t')
ds.visualize(floc_mask, space='fsaverage6', threshold=3)

In [ ]:
ds.flocER.plot_design(exclude_confs=True)

### Fit Responsefitter

In [ ]:
ds.flocER.shape_estimation(
    osf=30,
    TR=0.7,
    slice_time_ref=0.5,
    n_regressors=7,
    basis_set='fourier',
    interval=[0, 14],
    separate_conditions=True
)

In [ ]:
ds.plot_hrfs(taskset='flocER', rsq_cutoff=0.2)

Or plot them for the FFA (for face-HRFs only):

In [ ]:
ds.plot_hrfs(taskset='flocER', mask=floc_mask > 7, condition='face')

## Single-trial faces

In [ ]:
def filter_face(df):
    df_filt = df.query("event_type == 'face' & phase == 0").drop('trial_type', axis=1)
    df_filt['trial_type'] = 'face'
    return df_filt

def filter_expressive(df):
    df_filt = df = df.query("event_type == 'face' & phase == 0")
    return df_filt

In [ ]:
#ds.create_taskset(task='face', space='fsaverage5', acq='Mb4Mm27Tr700', ses=True, hemi='R')
#ds.face.preprocess(smoothing_fwhm=4, df_filter=filter_face)
#ds.face.glm_detect(hrf_model='glover', noise_model='ols', TR=.7)
#face_univar = ds.face.compute_fxe('face', stat_type='t')
ds.visualize(face_univar, space='fsaverage5', threshold=3, hemi='R', vmax=9)

In [ ]:
ds.face.glm_detect(hrf_model=rfv.loc[:, floc_mask > 4].values.mean(axis=1), noise_model='ols', TR=.7)
face_univar = ds.face.compute_fxe('face', stat_type='t')
ds.visualize(face_univar, space='fsaverage5', threshold=3, hemi='R', vmax=9)

## Check out different contrasts
E.g., face sex.

In [ ]:
def filter_face_sex(df):
    df_filt = df.query("event_type == 'face' & phase == 0").drop('trial_type', axis=1)
    df_filt = df_filt.rename(columns={'face_sex': 'trial_type'})
    return df_filt

In [ ]:
ds.face.preprocess(smoothing_fwhm=4, df_filter=filter_face_sex)
ds.face.glm_detect(hrf_model='glover', noise_model='ols', TR=.7)
face_sex_img = ds.face.compute_fxe('female - male', stat_type='t')
ds.visualize(face_sex_img, surface=True, threshold=3, hemi='R')

Or parametric modulation of attractiveness.

In [ ]:
def filter_face_attractiveness(df):
    df_filt = df.query("event_type == 'face' & phase == 0").drop('trial_type', axis=1)
    df_filt['trial_type'] = 'face'
    df_filt['modulation'] = 1
    df2 = df_filt.copy()
    mod = df2.loc[:, 'attractiveness'].values
    df2.loc[:, 'modulation'] = mod - mod.mean()
    df2['trial_type'] = 'face_attractiveness'
    df = pd.concat((df_filt, df2), axis=0)
    return df

In [ ]:
ds.face.preprocess(smoothing_fwhm=4, df_filter=filter_face_attractiveness)
ds.face.glm_detect(hrf_model='glover', noise_model='ols', TR=.7)
face_attractiveness_img = ds.face.compute_fxe('face_attractiveness', stat_type='t')
ds.visualize(face_attractiveness_img, surface=True, threshold=3, hemi='R')